2024-09-19 - Ronen Shekel

In [2]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots, get_imaging_masks
from pianoq_results.scan_result import ScanResult
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.misc.mplt import mimshow, mplot
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
import time 
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
U_no = 1
phases_no = 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# HW

In [3]:
# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got Zaber motors
Got Thorlabs motors
Got MPLC
Got Time tagger


# Single counts 

In [ ]:
resolution = 1 
filter_type = '20nm'
integration_time = 1 
tt.set_integration_time(integration_time)
scanner_sig, _, _, _ = get_signal_scanner(integration_time=integration_time, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)
scanner_idl, _, _, _ = get_idler_scanner(integration_time=integration_time, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)

masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_res_{resolution}_filter_{filter_type}.scan'
singles_signal_path = fr'{dir_path}\{timestamp}_signal_res_{resolution}_filter_{filter_type}.scan'

scanner_idl.saveto_path = singles_idler_path
scanner_sig.saveto_path = singles_signal_path
scanner_idl.scan(x_motor=mxi, y_motor=myi, ph=tt)    
scanner_sig.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 1404. Singles2: 2084. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 1280. Singles2: 2025. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 1392. Singles2: 1984. Coincidence: 0.00.
dur: 6. pix: 0, 3. Singles1: 1404. Singles2: 1979. Coincidence: 0.00.
dur: 8. pix: 0, 4. Singles1: 1371. Singles2: 2026. Coincidence: 0.00.
dur: 9. pix: 0, 5. Singles1: 1482. Singles2: 2083. Coincidence: 0.00.
dur: 11. pix: 0, 6. Singles1: 1507. Singles2: 2059. Coincidence: 0.00.
dur: 13. pix: 0, 7. Singles1: 1436. Singles2: 1992. Coincidence: 0.00.
dur: 15. pix: 0, 8. Singles1: 1423. Singles2: 1969. Coincidence: 0.00.
dur: 17. pix: 0, 9. Singles1: 1367. Singles2: 2073. Coincidence: 0.00.
dur: 19. pix: 0, 10. Singles1: 1447. Singles2: 2046. Coincidence: 0.00.


# correlations

In [12]:
integration_time = 30
filter_type = '80nm'

# in the different Us different phases this seemed to work best 
masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]
phases_path = glob.glob(rf'{dir_path}\U{phases_no}U*.phases')[0]

pr = PhaseFinderResult()
pr.loadfrom(phases_path)

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks

masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
masks = add_phase_input_spots(masks, pr.phases)
mplc.load_masks(masks, linear_tilts=True)

# locs 
locs_sig_path = glob.glob(fr'{dir_path}\U{U_no}U*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\U{U_no}U*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
scan_path = fr'{dir_path}\{timestamp}_filter_{filter_type}_exp_{integration_time}s.dscan'
mplc.restore_location()
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, no_hw_mode=True)
dps.zaber_ms = zaber_ms
dps.m_sig_x = mxs
dps.m_sig_y = mys
dps.m_idl_x = mxi
dps.m_idl_y = myi
dps.time_tagger = tt
dps.time_tagger.set_integration_time(integration_time)

dps.scan()

beginning scan
0, 0: 16450.77, 19548.33, 8.47
0, 1: 16418.57, 24280.53, 2.83
0, 2: 16502.97, 24620.47, 2.40
0, 3: 16509.43, 24081.40, 2.03
0, 4: 16475.10, 21247.50, 2.40
1, 0: 24732.30, 19432.23, 3.10
1, 1: 24738.87, 24208.17, 19.20
1, 2: 24868.70, 24643.83, 3.50
1, 3: 24718.70, 23976.17, 3.90
1, 4: 24698.90, 21188.77, 2.57
2, 0: 25333.23, 19361.53, 2.90
2, 1: 25419.73, 24141.07, 3.70
2, 2: 25416.77, 24509.40, 24.77
2, 3: 25724.27, 24161.73, 3.33
2, 4: 25528.10, 21317.43, 2.63
3, 0: 22759.67, 19499.77, 1.87
3, 1: 22696.70, 24275.33, 4.13
3, 2: 22894.87, 24685.07, 3.00
3, 3: 22701.30, 23959.37, 17.23
3, 4: 22764.83, 21252.40, 4.03
4, 0: 20036.90, 19524.20, 3.20
4, 1: 20088.80, 24287.07, 2.67
4, 2: 19845.23, 24418.57, 2.90
4, 3: 19771.47, 23797.40, 2.67
4, 4: 19825.37, 21090.97, 10.87


# Phases

In [ ]:
# masks 
masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
initiail_phases = np.zeros(25 * 2)


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_80nm.phases'
integration_time = 10
pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, coin_window=1e-9, no_hw_mode=True)

pf.zaber_ms = zaber_ms
pf.m_sig_x = mxs
pf.m_sig_y = mys
pf.m_idl_x = mxi
pf.m_idl_y = myi
pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_idl_path = glob.glob(fr'{dir_path}\U1U*idl.locs')[0]
locs_sig_path = glob.glob(fr'{dir_path}\U1U*sig.locs')[0]
locs_idl = np.load(locs_idl_path)['locs']
locs_sig = np.load(locs_sig_path)['locs']

# i,j of corr matrix that is supposed to be strong 
i = 1
j = 1
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

Plan: 
- run singles scan 
- reoptimize phases and try again correlations 